In [6]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")
PACKAGE_ROOT = os.path.dirname(os.path.abspath(""))
print(PACKAGE_ROOT)
sys.path.insert(0, PACKAGE_ROOT)
import joblib

import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
# from sklearn.model_selection import cross_val_predict
# from sklearn_crfsuite.metrics import flat_classification_report
from sklearn_crfsuite import CRF
# from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.metrics import classification_report

from preprocessing.crf_preprocessor import CRFPreprocessor
from utils.utils import get_connl_data
from data.ingestor import DataSaver, DataLoader

c:\Users\sharm\Documents\workspace\gitworks\ner-bilstm-crf


In [2]:
train_df = get_connl_data("train")
valid_df = get_connl_data("valid")
test_df = get_connl_data("test")

obj = CRFPreprocessor()
train_sentences = obj.get_sentences(train_df)
valid_sentences = obj.get_sentences(valid_df)
test_sentences = obj.get_sentences(test_df)

X_train, y_train = obj.preprocess(train_sentences)
X_valid, y_valid = obj.preprocess(valid_sentences)
X_test, y_test = obj.preprocess(test_sentences)

In [3]:
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [4]:
ds = DataSaver()
ds.save_model(model=crf, model_type="crf")

In [7]:
dl = DataLoader()
PATH = r"C:\Users\sharm\Documents\workspace\gitworks\ner-bilstm-crf\outputs\model_artifacts\crf-model-25-03-2023-17-21-01.pkl"
crf = joblib.load(PATH)

In [11]:
ds = DataSaver()
train_report_df = obj.inference(crf, X_train, y_train)
ds.save_report(train_report_df, "crf", "train")

valid_report_df = obj.inference(crf, X_valid, y_valid)
ds.save_report(valid_report_df, "crf", "valid")

test_report_df = obj.inference(crf, X_test, y_test)
ds.save_report(test_report_df, "crf", "test")

In [ ]:
# import eli5
# # eli5.show_weights(crf, top=30)

In [ ]:
# !pip install eli5
# import eli5

In [ ]:
# eli5.show_weights(crf, top=30)